In [364]:
import pandas as pd
import numpy as np
import random
import sqlite3 as sql
from datetime import datetime, timedelta
import os

In [365]:
conn = sql.connect("el_2.db")
cur = conn.cursor()

In [366]:
kpi = ["KPI1","KPI2","KPI3","KPI4"]
db_col = ["Index","EName",*kpi,"TimeStamp"]
weeks = ["Week1","Week2","Week3","CurrentHour"]
is_monday = True
d = "2024-09-15 22:00:00"
plant_data_col = ["Week","WeekStart","WeekEnd","IsUpdated",*kpi]

In [367]:
def truncate_table(cmd,name):
    try:
        q = f"DROP TABLE {name}" if cmd == 'Drop' else f"DELETE FROM {name}"
        cur.execute(q)
        conn.commit()
    except Exception as e:
        print(e)
        raise

In [368]:
# truncate_table('Drop','PlantData')

In [369]:
def get_date(date):
    currDate = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    return currDate

In [370]:
def fetch_data_within_range(start,end):
    try:
        data = cur.execute(f"SELECT EName, {",".join(kpi)}, TimeStamp from Transactions WHERE TimeStamp BETWEEN '{start}' AND '{end}';")
        selected_data = data.fetchall()
        df = pd.DataFrame(selected_data,columns=["EName",*kpi,"TimeStamp"])
        return df
    except sql.Error as e:
        print(e)
        return []

In [371]:
def get_current_week_range(current_date):
    current_date = current_date.replace(hour=0, minute=0, second=0, microsecond=0) 
    current_week_start = current_date - timedelta(days=current_date.weekday()) if is_monday else current_date
    current_week_end = current_week_start  + timedelta(days=6, hours=23, minutes=59, seconds=59)
    return current_week_start,current_week_end

In [372]:
def get_prev_week_range(curr_week_start):
    curr_week_start = curr_week_start.replace(hour=0, minute=0, second=0, microsecond=0)
    prev_week_start = curr_week_start - timedelta(weeks=1)
    prev_week_end = curr_week_start - timedelta(seconds=1)
    return prev_week_start,prev_week_end

In [373]:
def get_current_hour_range(current_hour):
    current_hour = current_hour.replace(minute=0, second=0, microsecond=0)
    prev_hour_end = current_hour - timedelta(hours=1)
    return current_hour,prev_hour_end

In [374]:
def table_exists(table_name):
    query = "SELECT * FROM sqlite_master WHERE type='table' AND name=?;"
    cur.execute(query, (table_name,))
    result = cur.fetchone()
    return result is not None

In [375]:
def generate_plant_data():
    cols = []
    for i in kpi:
        cols.append(i + " REAL Default 0")
    query1 = f'''
        CREATE TABLE IF Not Exists PlantData
        (
            Id INTEGER PRIMARY KEY,
            Week TEXT NOT NULL,
            WeekStart DATETIME DEFAULT "1999-01-01 00:00:00",
            WeekEnd DATETIME DEFAULT "1999-01-01 00:00:00",
            IsUpdated BOOLEAN DEFAULT FALSE,
            {", ".join(cols)}
        )
    '''
    
    w = []
    for i in weeks:
        w.append(f"('{i}')")

    query2 = f'''
        INSERT INTO PlantData (Week) VALUES {", ".join(w)}
    '''

    try:
        cur.execute(query1)
        print("Table created")
        cur.execute(query2)
        conn.commit()
        print("Data inserted")
    except Exception as e:
        print(e)
        conn.rollback()
        raise


In [376]:
def check_week_data():
    query = f'''SELECT IsUpdated, {", ".join(kpi)} FROM PlantData'''
    row = cur.execute(query)
    rows = row.fetchall()
    df = pd.DataFrame(rows,columns=["IsUpdated",*kpi])
    return True if df["IsUpdated"].sum() >= len(df) else False

In [377]:
def calculate_mean(data):
    kpi_avg = np.round((data[kpi].mean()).infer_objects(copy=False).fillna(0),5).tolist()
    return kpi_avg

In [378]:
def fetch_weekly_data(d):
    print("Fetching week data")
    df = pd.DataFrame(columns=plant_data_col)
    current_week_start,_= get_current_week_range(d)
    for i in weeks:
        if i != "CurrentHour":
            data_l = []
            prev_week_start,prev_week_end = get_prev_week_range(current_week_start)
            print(f"Fetching {i}",prev_week_start,prev_week_end)
            prev_week_data = fetch_data_within_range(prev_week_start,prev_week_end)
            calculated_mean = calculate_mean(prev_week_data)
            data_l.extend([i,prev_week_start,prev_week_end,True])
            data_l.extend(calculated_mean)
            current_week_start = prev_week_start
            df.loc[len(df)] = data_l
            print(f"{i} fetched")
    return df

def fetch_hourly_data(d):
    df = pd.DataFrame(columns=plant_data_col)
    data_l = []
    current_hour, prev_hour_start = get_current_hour_range(d)
    current_week_start,current_week_end= get_current_week_range(d)
    _,prev_week_end = get_prev_week_range(current_week_start)
    print("Fetching hourly data",current_hour, prev_hour_start)
    curr_hour_data = fetch_data_within_range(prev_hour_start,current_hour)
    calculated_mean = calculate_mean(curr_hour_data)
    data_l.extend(['CurrentHour',current_week_start,current_week_end,True])
    data_l.extend(calculated_mean)
    df.loc[len(df)] = data_l
    return df

In [379]:
def current_h():
    try:
        q = '''SELECT * FROM PlantData WHERE WEEK = "CurrentHour"'''
        data = cur.execute(q)
        row = data.fetchone()
        if row and len(row) > 0:
            end_date = pd.Series(row)
            return end_date[3]
        return ""
    except sql.Error as e:
        print(e)
        raise

In [380]:
def is_weekend(d):
    current_hour, _ = get_current_hour_range(d)
    db_hour = current_h()
    # print(db_hour)
    curr_weekend = get_date(db_hour)
    return current_hour >= curr_weekend

In [381]:
def get_data_hourly(d):
    d = get_date(d)
    
    if not table_exists("PlantData"):
        generate_plant_data()
    
    curr_df = fetch_hourly_data(d)
    # print(type(d))
    print(is_weekend(d))
    
    if is_weekend(d) or not check_week_data():
        week_df = fetch_weekly_data(d)
        df = pd.concat([week_df,curr_df],ignore_index=True)
        return df
    return curr_df

In [382]:
def dump_data(df):
    try:
        if len(df) > 1:
            query_1 = 'DELETE FROM PlantData'
            cur.execute(query_1)
            df.to_sql("PlantData",conn,if_exists='replace')
        else:
            query_1 = '''
                DELETE FROM PlantData
                WHERE Week = 'CurrentHour';
            '''
            cur.execute(query_1)
            df.to_sql('PlantData',conn,if_exists='append')
        conn.commit()
    except sql.Error as e:
        print(e)
        conn.rollback()
        raise


In [383]:
bar_df = get_data_hourly(d)

Fetching hourly data 2024-09-16 00:00:00 2024-09-15 23:00:00
True
Fetching week data
Fetching Week1 2024-09-09 00:00:00 2024-09-15 23:59:59
Week1 fetched
Fetching Week2 2024-09-02 00:00:00 2024-09-08 23:59:59
Week2 fetched
Fetching Week3 2024-08-26 00:00:00 2024-09-01 23:59:59
Week3 fetched
Fetching Week4 2024-08-19 00:00:00 2024-08-25 23:59:59
Week4 fetched


In [384]:
bar_df.head()

,Week,WeekStart,WeekEnd,IsUpdated,KPI1,KPI2
0,Week1,2024-09-09,2024-09-15 23:59:59,True,51.50074,31.50004
1,Week2,2024-09-02,2024-09-08 23:59:59,True,51.50016,31.50028
2,Week3,2024-08-26,2024-09-01 23:59:59,True,51.49819,31.50025
3,Week4,2024-08-19,2024-08-25 23:59:59,True,0.00000,0.00000
4,CurrentHour,2024-09-16,2024-09-22 23:59:59,True,51.51332,31.51600


In [385]:
dump_data(bar_df)

In [386]:
def select_bar_data():
    try:
        _ = '''SELECT * FROM PlantData;'''
        rows = cur.execute(_)
        data = rows.fetchall()
        df = pd.DataFrame(data,columns=["Index",*plant_data_col])
        return df
    except sql.Error as e:
        print(e)
        raise


In [387]:
def convert_string(l):
    temp = []
    for i in l:
        temp.append(str(i))
    l_str = ", ".join(temp)
    return str(l_str)

In [1]:
def fetch_bar_data():
    db_data = select_bar_data()
    bar_data = db_data.tail(len(weeks))
    bar_data_dict = {
        'x':", ".join(weeks),
        'y':", ".join(kpi)
    }
    for i in range(0,len(weeks)):
        bar_data_dict[f"{weeks[i]}"] = convert_string(bar_data[[*kpi]].loc[i].tolist())

    return bar_data_dict

In [389]:
fetch_bar_data()

{'x': 'Week1,Week2,Week3,Week4,CurrentHour',
 'Week1': '51.50074, 31.50004',
 'Week2': '51.50016, 31.50028',
 'Week3': '51.49819, 31.50025',
 'Week4': '0.0, 0.0',
 'CurrentHour': '51.51332, 31.516'}